In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob



CHECKERBOARD = (7,7)

points3dL = []
points2dL = []

points3dR = []
points2dR = []

objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

chessboard_images_left = []
chessboard_images_right = []
for i in range(1,5):
    chessboard_images_left.append(cv2.imread('chessboards/c' + str(i) + 'Left.png'))
    chessboard_images_right.append(cv2.imread('chessboards/c' + str(i) + 'Right.png'))

for img in chessboard_images_left:

    grayL = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(grayL, CHECKERBOARD, None)
    if ret:
        #img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        points3dL.append(objp)
        points2dL.append(corners)

for img in chessboard_images_right:

    grayR = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(grayR, CHECKERBOARD, None)
    if ret:
        #img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        points3dR.append(objp)
        points2dR.append(corners)
    #cv2.imshow('img',img)
    #cv2.waitKey(0)

# cv2.destroyAllWindows()


 

h,w = img.shape[:2]
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(points3dL, points2dL, grayL.shape[::-1], None, None)

opticalCenterL= np.array(mtxL[0][2], mtxL[1][2]) #cx, cy
focalLengthL = np.array(mtxL[0][0], mtxL[1][1]) #fx, fy

# #find optical center of left camera
# optical_centerL = np.array([0,0,0])
# optical_centerL = np.dot(mtxL, optical_centerL)
# optical_centerL = np.dot(np.linalg.inv(rvecsL[0]), optical_centerL - tvecsL[0])
# optical_centerL = np.dot(np.linalg.inv(mtxL), optical_centerL)
# optical_centerL = optical_centerL / optical_centerL[2]


retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(points3dR, points2dR, grayR.shape[::-1], None, None)

opticalCenterR= np.array(mtxR[0][2], mtxR[1][2]) #cx, cy
focalLengthR = np.array(mtxR[0][0], mtxR[1][1]) #fx, fy

# for point in points2dR[-1]:
#     grayR[-1][point@distR[-1]] = [255,0,0]
#     cv2.imshow('img',grayR[-1])
#     cv2.waitKey(0)
# cv2.destroyAllWindows()


#stereo calibration using the left and right camera matrices and distortion coefficients
ret, mtxL, distL, mtxR, distR, R, T, E, F = cv2.stereoCalibrate(points3dL, points2dL, points2dR, mtxL, distL, mtxR, distR, grayL.shape[::-1]) #", None, None, None, None, cv2.CALIB_FIX_INTRINSIC, (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))"


# optical centre = inv(rot)*-trans

# maybe use triangulate point


# print("Camera matrix : \n")
# print(mtxL)
# print("dist : \n")
# print(distL)
# print("rvecs : \n")
# print(rvecsL)
# print("tvecs : \n")
# print(tvecsL)

#use stereoRectify to calculate the rectification transforms
rectify_scale = 0 #alpha
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtxL, distL, mtxR, distR, grayL.shape[::-1], R, T, rectify_scale, (0,0)) # last paramater is alpha, if 0= crop, if 1= all pixels

print(F)

[[-3.93215154e-08  7.82607265e-07 -1.96522540e-04]
 [ 1.05197628e-06  4.40299094e-08 -1.24505789e-02]
 [-8.53593685e-04  1.07493969e-02  1.00000000e+00]]


In [14]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob


F = np.array([[-3.93215154e-08,  7.82607265e-07, -1.96522540e-04],[ 1.05197628e-06,  4.40299094e-08, -1.24505789e-02],[-8.53593685e-04 , 1.07493969e-02 , 1.00000000e+00]])

chimp_left = []
chimp_right = []
chimp_left_b = []
chimp_right_b = []

chimp_left = glob.glob('scanLeft/*.png')
chimp_right = glob.glob('scanRight/*.png')

for fname in chimp_left:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    chimp_left_b.append(cv2.inRange(gray,30,160))

for fname in chimp_right:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    chimp_right_b.append(cv2.inRange(gray,30,160))


#print(chimp_left_b[11])

#----------------------------------Test F
imgt = chimp_left_b[11]
imgtr = cv2.imread(chimp_right[11])
imgr = cv2.imread(chimp_left[11])
#cv2.imshow('img', imgt)
# cv2.waitKey(0)

print(imgt.shape)
# imgr[imgt==255] = [0,255,0]
ind = np.where(imgt == 255)
print(ind)

i=0
for y,x in zip(ind[0],ind[1]):
    if i ==200:
        lp = F@np.array([[x],[y],[1]])
        start = -(lp[0]*100+lp[2])/lp[1]
        stop= -(lp[0]*1200+lp[2])/lp[1]
        imgtr = cv2.line(imgtr, (100,int(start)), (1200,int(stop)), (0,255,0), 1)
        i = 0
    i+=1

# cv2.imshow('img', imgtr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# y = ind[0][40]
# x = ind[1][40]
# lp = F@np.array([[x],[y],[1]])
# print(lp)
# start = -(lp[0]*100+lp[2])/lp[1]
# stop= -(lp[0]*1200+lp[2])/lp[1]
# print(start)
# print(stop)



# #cv2.imshow('img',cv2.line(imgtr, (0,int(start[int(len(start)/2)])), (1079,int(stop[int(len(start)/2)])), (0,255,0), 10) )
# cv2.imshow('img',cv2.line(imgtr, (100,int(start)), (1200,int(stop)), (0,255,0), 10) )
# #imgtr[(500,int(start))] = [0,255,0]
# cv2.imshow('img', imgtr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()




#-----------------------------------------


#------------------------------------test f cv2
imgl = chimp_left_b[11]
imgr = chimp_right_b[11]
imgtr = cv2.imread(chimp_right[11])
imgtl = cv2.imread(chimp_left[11])

pointsLeft = np.argwhere(imgl == 255)
pointsRight = np.argwhere(imgr == 255)


reducedLeft = {}
nb = {}
for i in range(len(pointsLeft)):
    pointLeft = pointsLeft[i]
    if pointLeft[0] in reducedLeft.keys():
        reducedLeft[pointLeft[0]] += pointLeft[1]
        nb[pointLeft[0]] += 1
    else:
        reducedLeft[pointLeft[0]] = pointLeft[1]
        nb[pointLeft[0]] = 1

for key in reducedLeft.keys():
    reducedLeft[key] = int(reducedLeft[key]/nb[key])

# for i,j in reducedLeft.items():
#     imgtl[i,j] = [0,255,0]

# cv2.imshow('img', imgtl)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


pointsLeft = np.argwhere(imgl == 255)


k=0



for i,j in reducedLeft.items():
    #pointLeft = pointsLeft[j]
    pointLeft = np.array([j, i])
    pointLeft = np.array([pointLeft], dtype=np.float32)
    lines = cv2.computeCorrespondEpilines(pointLeft, 1, F)
    #print("aaa")
    #print(lines)
    lines = lines.reshape(-1,3)
    print(lines)

    # draw epiline on right image in white color and display it
    x0, y0 = map(int, [0, -lines[0][2]/lines[0][1] ])
    x1, y1 = map(int, [imgtr.shape[1], -(lines[0][2]+lines[0][0]*imgtr.shape[1])/lines[0][1] ])
    if k ==2:
        imgtr = cv2.line(imgtr, (x0, y0), (x1, y1), (0, 255, 0), 1)
        k = 0

    k+=1


reducedRight = {}
nb = {}
for i in range(len(pointsRight)):
    pointRight = pointsRight[i]
    #pointRight = np.array([pointRight[1], pointRight[0]])
    if pointRight[0] in reducedRight.keys():
        reducedRight[pointRight[0]] += pointRight[1]
        nb[pointRight[0]] += 1
    else:
        reducedRight[pointRight[0]] = pointRight[1]
        nb[pointRight[0]] = 1

for key in reducedRight.keys():
    reducedRight[key] = int(reducedRight[key]/nb[key])

for i,j in reducedRight.items():
    imgtr[i,j] = [0,255,0]

cv2.imshow('img', imgtr)
cv2.waitKey(0)
cv2.destroyAllWindows()

#compute distance between a line and a point
def distPointLine(point, line):
    return abs(line[0]*point[0]+line[1]*point[1]+line[2])/np.sqrt(line[0]**2+line[1]**2)

best_points=[]
for line in lines:
    minDist = float("inf")
    for i,j in reducedRight.items():
        point = np.array([j, i])
        dist = distPointLine(point, line)
        if dist < minDist:
            minDist = dist
            best_point = point
    best_points.append(best_point)

linesL = []
linesR = []

for i in range(len(lines)):
    pointL = reducedLeft.items[i]
    pointR = best_points[i]
    focalLengthL = focalLengthL[0]
    focalLengthR = focalLengthR[0]
    









#-----------------------------------------


#--------------------dump

# for j in range(len(pointsLeft)):
#     pointLeft = pointsLeft[j]
#     pointLeft = np.array([pointLeft[1], pointLeft[0]])
#     pointLeft = np.array([pointLeft], dtype=np.float32)
#     lines = cv2.computeCorrespondEpilines(pointLeft, 1, F)
#     lines = lines.reshape(-1,3)

#     # draw epiline on right image in white color and display it
#     x0, y0 = map(int, [0, -lines[0][2]/lines[0][1] ])
#     x1, y1 = map(int, [imgtr.shape[1], -(lines[0][2]+lines[0][0]*imgtr.shape[1])/lines[0][1] ])
#     if i ==2:
#         imgtr = cv2.line(imgtr, (x0, y0), (x1, y1), (0, 255, 0), 1)
#         i = 0

#     i+=1

#--------------------------



# points = []
# for imgL, imgR in zip(chimp_left_b, chimp_right_b):
#     stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
#     disparity = stereo.compute(imgL,imgR).astype(np.float32) / 16.0
#     points.append(cv2.reprojectImageTo3D(disparity, Q))
#     plt.imshow(disparity, cmap="gray")



# #projetc to 3D
# #D = cv2.reprojectImageTo3D(disparity, Q)

# #plot 3d points
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for point in points:
#     ax.scatter(point[:,:,0], point[:,:,1], point[:,:,2], c='r', marker='.')
# plt.show()

# for imgL, imgR in zip(chimp_left_b, chimp_right_b):
#     arrayL = np.argwhere(imgL!=0)
#     arrayR = np.argwhere(imgR!=0)
#     for pointL in arrayL:
#         pointL = np.hstack((pointL,1))
#         res = []
#         for pointR in arrayR:
#             pointR = np.hstack((pointR,1))
#             res.append(pointL.T@F@pointR)
#         print(min(res))
            



(1080, 1920)
(array([128, 128, 128, ..., 898, 898, 899], dtype=int64), array([1137, 1138, 1139, ..., 1168, 1169, 1166], dtype=int64))
aaa
[[-1.2543402e-02 -9.9992132e-01  1.2486205e+02]]
aaa
[[-1.2487876e-02 -9.9992204e-01  1.2562585e+02]]
aaa
[[-1.24230012e-02 -9.99922812e-01  1.26518265e+02]]
aaa
[[-1.2358112e-02 -9.9992365e-01  1.2741085e+02]]
aaa
[[-1.2293211e-02 -9.9992442e-01  1.2830362e+02]]
aaa
[[-1.2223656e-02 -9.9992532e-01  1.2926041e+02]]
aaa
[[-1.2158735e-02 -9.9992609e-01  1.3015344e+02]]
aaa
[[-1.2089171e-02 -9.9992692e-01  1.3111034e+02]]
aaa
[[-1.2024230e-02 -9.9992770e-01  1.3200365e+02]]
aaa
[[-1.1954660e-02 -9.9992853e-01  1.3296063e+02]]
aaa
[[-1.1885087e-02 -9.9992937e-01  1.3391765e+02]]
aaa
[[-1.1820119e-02 -9.9993014e-01  1.3481133e+02]]
aaa
[[-1.1750539e-02 -9.9993098e-01  1.3576843e+02]]
aaa
[[-1.1680959e-02 -9.9993175e-01  1.3672556e+02]]
aaa
[[-1.1611377e-02 -9.9993259e-01  1.3768269e+02]]
aaa
[[-1.1541795e-02 -9.9993336e-01  1.3863983e+02]]
aaa
[[-1.147678